In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
from ydata_profiling import ProfileReport

In [3]:
root_path = Path(os.getcwd()).resolve().parent
sys.path.append(str(root_path))

In [4]:
from auto_importer import DataFrameDescriber

In [5]:
data = pd.read_parquet('train.parquet').set_index('laptop_ID')

In [6]:
df = data.copy()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 599 to 87
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    object 
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_euros       912 non-null    float64
dtypes: float64(1), object(11)
memory usage: 92.6+ KB


In [8]:
complete_describe = DataFrameDescriber(df)

In [9]:
complete_describe.describe_categorico()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
count,912,912,912,912,912,912,912,912,912,912,912
unique,19,484,6,18,38,104,9,34,94,9,160
top,Dell,XPS 13,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.2kg
freq,214,21,498,461,357,137,436,282,198,760,84


In [10]:
complete_describe.describe_numerico()

,Price_euros
count,912.000000
mean,1111.262533
std,682.109694
min,174.000000
25%,598.742500
50%,977.000000
75%,1472.650000
max,5499.000000


In [11]:
df.columns

Index(['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros'],
      dtype='object')

In [12]:
report = ProfileReport(df)

In [13]:
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
laptop_ID,,,,,,,,,,,,
599,Dell,Inspiron 3179,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Core M 7Y30 1.0GHz,4GB,128GB SSD,Intel HD Graphics 615,Windows 10,1.39kg,603.0
92,Asus,FX550IK-DM018T (FX-9830P/8GB/1TB/Radeon,Gaming,15.6,Full HD 1920x1080,AMD FX 9830P 3GHz,8GB,1TB HDD,AMD Radeon RX 560,Windows 10,2.45kg,699.0
744,HP,ZBook 15,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD,Nvidia Quadro M2200,Windows 10,2.6kg,2419.0
729,HP,Pavilion X360,2 in 1 Convertible,13.3,IPS Panel Touchscreen 1366x768,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.58kg,836.6
957,Toshiba,Tecra A50-C-1ZV,Notebook,15.6,1366x768,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,2.2kg,1119.0


Inicio limpieza de datos

In [15]:
#Eliminando la unidad de medida y dejando solo el valor numérico para transformar la columna de tipo objeto a numero flotante

df['Weight'] = df['Weight'].str.replace('kg','').astype(float)

In [16]:
#Eliminando la unidad de medida y dejando solo el valor numérico para transformar la columna de tipo objeto a numero intero

df['Ram'] = df['Ram'].str.replace('GB','').astype(int)

In [17]:
# Transformación columna inches de tipo objeto a numero flotante

df['Inches'] = df['Inches'].astype(float)

In [18]:
# list_a = []
# list_b = []
# for cpu in df['Cpu']:
#     list_a.append(cpu.split()[1:-1])

# for output_model_cpu in list_a:
#     list_b.append(' '.join(output_model_cpu))

In [19]:
# Creando lista con el nombre de la fabricante de la CPU

cpu_brand_list = []
for cpu in df['Cpu']:
    cpu_brand_list.append(cpu.split()[0])

list_a = []
cpu_model_list = []
for cpu in df['Cpu']:
    list_a.append(cpu.split()[1:-1])

for output_model_cpu in list_a:
    cpu_model_list.append(' '.join(output_model_cpu))

cpu_freq_list = []
for cpu in df['Cpu']:
    cpu_freq_list.append(cpu.split()[-1].replace('GHz',''))

In [20]:
# Creando columna en el DF para recibir los valores que provienen de la lista anterior

df.insert(7,'CPU_Brand', cpu_brand_list)
df.insert(8,'CPU_Model', cpu_model_list)
df.insert(9,'CPU_Freq', cpu_freq_list)

In [21]:
#Eliminando la unidad de medida y dejando solo el valor numérico para transformar la columna de tipo objeto a numero intero

df['CPU_Freq'] = df['CPU_Freq'].astype(float)

In [22]:
# Drop columna CPU que dio origen a tres nuevas columnas que describen más específicamente cada procesador

df = df.drop(columns='Cpu')

In [23]:
# Creación de las columnas que recibirán los valores de resolución de pantalla

df[['Resolution_X', 'Resolution_Y']] = df['ScreenResolution'].str.extract(r'(\d+)x(\d+)').astype(int)

In [24]:
# Posicionamento de las columnas en sus lugares debidos

df.insert(5, 'Resolution_X', df.pop('Resolution_X'))
df.insert(6, 'Resolution_Y', df.pop('Resolution_Y'))

In [25]:
# Creación de la columna Is_touch que asigna valor 1 para laptop con pantalla touchscreen y 0 para pantallas no touchscreen

df['Is_touch'] = df['ScreenResolution'].str.contains('Touch', case=False).astype(int)

In [26]:
# Posicionamento de la columna en su lugar debido

df.insert(7, 'Is_touch', df.pop('Is_touch'))

In [27]:
# # Eliminación de la columna inicial de resolución de pantalla que fue reemplazada por las columnas recién creadas

df = df.drop(columns='ScreenResolution')

In [28]:
# Cambio del valor en la columna TypeName para laptops 2 en 1

df.loc[df['TypeName'].str.contains('Convertible', case=False, na=False), 'TypeName'] = 'Convertible'

In [29]:
# Arreglando los valores de los discos duros híbridos
df['Memory'] = df['Memory'].str.replace('1.0TB','1TB')

# Creación de las columnas de discos duros
df['SSD'] = df['Memory'].str.extract(r'(\d+)GB SSD').fillna(0).astype(int)
df['HDD'] = df['Memory'].str.extract(r'(\d+)TB HDD').fillna(0).astype(int) * 1024
df['Hybrid'] = df['Memory'].str.extract(r'(\d+)TB Hybrid').fillna(0).astype(int) * 1024
df['Flash_storage'] = df['Memory'].str.extract(r'(\d+)GB Flash Storage').fillna(0).astype(int)

# Posicionamento de las columnas en sus lugares debidos
df.insert(11, 'SSD', df.pop('SSD'))
df.insert(12, 'HDD', df.pop('HDD'))
df.insert(13, 'Hybrid', df.pop('Hybrid'))
df.insert(14, 'Flash_storage', df.pop('Flash_storage'))

# Eliminación de la columna inicial de los datos de disco duro que fue reemplazada por las nuevas columnas recién creadas
df = df.drop(columns='Memory')

In [30]:
# Creación de la lista con los nombres de la marca del GPU

gpu_brand = []
for gpu in df['Gpu']:
    gpu_brand.append(gpu.split()[0])

# Creación de la lista con los nombres de modelos del GPU
gpu_model = []
gpu_model_list = []
for gpu in df['Gpu']:
    gpu_model.append(gpu.split()[1:])

for output_model_gpu in gpu_model:
    gpu_model_list.append(' '.join(output_model_gpu))

In [31]:
# Creación de las nuevas columnas posicionándolas en sus lugares debidos

df.insert(16,'GPU_Brand', gpu_brand)
df.insert(17,'GPU_Model', gpu_model_list)

In [32]:
# Creación y posicionamento de la columna que informa se el GPU es dedicado o integrado

df['Is_GPU_dedicated'] = np.where(df['GPU_Brand'].str.contains('Intel', case=False),0,1).astype(int)

df.insert(18, 'Is_GPU_dedicated', df.pop('Is_GPU_dedicated'))

In [33]:
# Eliminación de la columna inicial GPU que fue reemplazada por las nuevas columnas recién creadas

df = df.drop(columns='Gpu')

In [34]:
# Arreglando valores de la columna OpSys excluyendo las versiones y dejando solo el nombre de la empresa fabricante del OS en uso

def categorize_os(os):
    if 'Windows' in os:
        return 'Windows'
    elif 'macOS' in os:
        return 'macOS'
    elif 'Mac OS' in os:
        return 'macOS'
    elif 'Linux' in os:
        return 'Linux'
    else:
        return 'Chrome OS'

df['OpSys'] = df['OpSys'].apply(categorize_os)

In [35]:
# pd.set_option('display.max_rows',None)
# df

In [36]:
df.head()

,Company,Product,TypeName,Inches,Resolution_X,Resolution_Y,Is_touch,Ram,CPU_Brand,CPU_Model,...,SSD,HDD,Hybrid,Flash_storage,GPU_Brand,GPU_Model,Is_GPU_dedicated,OpSys,Weight,Price_euros
laptop_ID,,,,,,,,,,,,,,,,,,,,,
599,Dell,Inspiron 3179,Convertible,11.6,1366,768,1,4,Intel,Core M 7Y30,...,128,0,0,0,Intel,HD Graphics 615,0,Windows,1.39,603.0
92,Asus,FX550IK-DM018T (FX-9830P/8GB/1TB/Radeon,Gaming,15.6,1920,1080,0,8,AMD,FX 9830P,...,0,1024,0,0,AMD,Radeon RX 560,1,Windows,2.45,699.0
744,HP,ZBook 15,Workstation,15.6,1920,1080,0,16,Intel,Core i7 7700HQ,...,256,0,0,0,Nvidia,Quadro M2200,1,Windows,2.60,2419.0
729,HP,Pavilion X360,Convertible,13.3,1366,768,1,8,Intel,Core i5 7200U,...,256,0,0,0,Intel,HD Graphics 620,0,Windows,1.58,836.6
957,Toshiba,Tecra A50-C-1ZV,Notebook,15.6,1366,768,0,8,Intel,Core i5 6200U,...,256,0,0,0,Intel,HD Graphics 520,0,Windows,2.20,1119.0


In [47]:
from sklearn.preprocessing import LabelEncoder

In [48]:
lb = LabelEncoder()

In [49]:
df['Company_enc'] = lb.fit_transform(df['Company'])

In [53]:
df['Company'].nunique()

19

In [50]:
df

,Company,Product,TypeName,Inches,Resolution_X,Resolution_Y,Is_touch,Ram,CPU_Brand,CPU_Model,...,HDD,Hybrid,Flash_storage,GPU_Brand,GPU_Model,Is_GPU_dedicated,OpSys,Weight,Price_euros,Company_enc
laptop_ID,,,,,,,,,,,,,,,,,,,,,
599,Dell,Inspiron 3179,Convertible,11.6,1366,768,1,4,Intel,Core M 7Y30,...,0,0,0,Intel,HD Graphics 615,0,Windows,1.39,603.00,4
92,Asus,FX550IK-DM018T (FX-9830P/8GB/1TB/Radeon,Gaming,15.6,1920,1080,0,8,AMD,FX 9830P,...,1024,0,0,AMD,Radeon RX 560,1,Windows,2.45,699.00,2
744,HP,ZBook 15,Workstation,15.6,1920,1080,0,16,Intel,Core i7 7700HQ,...,0,0,0,Nvidia,Quadro M2200,1,Windows,2.60,2419.00,7
729,HP,Pavilion X360,Convertible,13.3,1366,768,1,8,Intel,Core i5 7200U,...,0,0,0,Intel,HD Graphics 620,0,Windows,1.58,836.60,7
957,Toshiba,Tecra A50-C-1ZV,Notebook,15.6,1366,768,0,8,Intel,Core i5 6200U,...,0,0,0,Intel,HD Graphics 520,0,Windows,2.20,1119.00,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,Acer,Chromebook 11,Netbook,11.6,1366,768,0,4,Intel,Celeron Dual Core N3060,...,0,0,32,Intel,HD Graphics 400,0,Chrome OS,1.35,355.00,0
626,Acer,CB5-132T-C9KK (N3160/4GB/32GB/Chrome,Convertible,11.6,1366,768,1,4,Intel,Celeron Quad Core N3160,...,0,0,32,Intel,HD Graphics 400,0,Chrome OS,1.25,379.00,0
225,Dell,Inspiron 3576,Notebook,15.6,1920,1080,0,8,Intel,Core i5 8250U,...,0,0,0,AMD,Radeon 520,1,Linux,2.20,677.35,4
